In [5]:
import os 
import re
import codecs
import _uniout 
import glob
list1 = [u'壹',u'貳',u'參',u'肆']
list2 = [u'一',u'二',u'三',u'四']
list3 = [u'甲',u'乙',u'丙',u'丁']
for filename in glob.glob(u'./judicial/臺灣新竹,100,婚,118.txt'):
    with codecs.open( filename ,'r' ,'utf-8') as f :
        content = re.sub(u'[\r\n]',' ',f.read())
        content = re.sub(u'[ ]','',content)
        f.seek(0,0) #檔案指標移回檔案開頭
        lines = f.readlines()# 不能圖超過 2Mb
        print lines[0]
        cnt = 0
        lv = 0
        try:
            for i in lines:
                if lv != 0:
                    break
                m = re.search(u'([壹貳參肆一二三四甲乙丙丁]{1,2})、(原告.*?)',i)
                if m !=None:
                    n = m.group(1)
                    if n in list1:
                        lv = 1
                    if n in list2:
                        lv = 2
                    if n in list3:
                        lv = 3
                    for j in lines:
                        print lines[cnt]
                        cnt += 1
                        if lv == 1 and re.match(u'[壹貳參肆]{1,2}、.*?',lines[cnt]) !=None:
                            break
                        if lv == 2 and re.match(u'[一二三四]{1,2}、.*?',lines[cnt]) !=None:
                            break
                        if lv == 3 and re.match(u'[甲乙丙丁]{1,2}、.*?',lines[cnt]) !=None:
                            break
                else:
                    cnt += 1
        except:
            pass
        
    print '============================================================='

【裁判字號】100,婚,118

甲、原告方面：

一、聲明：准原告與被告離婚、訴訟費用由被告負擔。

二、原告起訴主張：兩造於民國94年12月15日結婚，婚後因個性

    不合，被告都要原告聽他的，常常吵架，還會動手打原告，

    原告已離家一年多，兩造已無法共同生活，爰依民法第1052

    條第2項訴請離婚等語。

